# Applied Physics Lab - III. Liquid Scintillation Spectroscopy @ ELTE

In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

In [ ]:
PC = True
if PC:
    FSS_dir = 'D://GitHub/ELTE_Applied_Physics_2019/III. Liquid Scintillation Spectroscopy/'
    FSS_data = FSS_dir + 'data/'
    figsave_dir = FSS_dir + 'Documentation/report/src/images/'
else:
    FSS_dir = 'C://Users/masterdesky/Documents/GitHub/ELTE_Applied_Physics_2019/III. Liquid Scintillation Spectroscopy/'
    FSS_data = FSS_dir + 'data/'
    figsave_dir = FSS_dir + 'Documentation/report/src/images/'

figsave_format = 'pdf'
figsave_dpi = 200
    
# Bold print
b1 = '\033[1m'
b0 = '\033[0m'

### $^{14}$C data

In [ ]:
Z = 6
# Q-value [keV]
Q = 156.5
DPM = 120900

In [ ]:
# mass of electron [kg]
m_e = 9.10938356e-31
# light speed [m/s]
c_light = 299792458
# 1 J = 6.2415096471204 * 10^{15} keV
J_to_keV = 6.2415096471204e15

print(b1 + 'm*c^2 = ' + b0 + '{0} keV'.format(m_e*c_light**2 * J_to_keV))

## Import data

In [ ]:
os.listdir(FSS_data)

In [ ]:
def read_data_files(file):
    
    bins = {}
    
    with open(file, mode='rt') as f:
        for line in f:

            # Dataline format:
            # "bin;v;v;v;..."
            # 
            s = line.split(';')
            # The first element will contain the bin label, while
            # the other elements contains the bin values
            # s[0] == bin
            # s[i != 0] == values
            s[0] = s[0].replace(',', '.')
            bins[float(s[0])-0.25] = [int(x) for x in s[1:]]

    # Convert to pandas DataFrame
    df = pd.DataFrame(data=bins)
    
    return df

In [ ]:
data_1 = read_data_files(file=(FSS_data + os.listdir(FSS_data)[1]))
data_2 = read_data_files(file=(FSS_data + os.listdir(FSS_data)[2]))
data_3 = read_data_files(file=(FSS_data + os.listdir(FSS_data)[3]))
data = data_1.append(data_2, ignore_index=True).append(data_3, ignore_index=True)
data = data.T

In [ ]:
data

### Drop errorous measurements

In [ ]:
for i in np.where(data.iloc[0].values > 100)[0]:
    del data[i]

data.columns = [i for i in range(len(data.columns.values))]

In [ ]:
data

### Calculate statistical data columns

In [ ]:
data['mean'] = [np.mean(data.iloc[i]) for i in range(len(data.index))]
data['std'] = [np.std(data.iloc[i]) for i in range(len(data.index))]

In [ ]:
data

## Plot spectrums

In [ ]:
nrows = 8
ncols = 3
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))
fig.subplots_adjust(hspace=0.45)

titlesize = 25
axislabelsize = 25
axisticksize = 20

random_indeces = np.random.randint(low=0, high=(len(data.columns)-2), size=nrows*ncols)
random_indeces = sorted(random_indeces)
for i in range(nrows):
    for j in range(ncols):
        axes[i][j].plot(data.index, data[random_indeces[i*ncols + j]],
                        lw=4, color='tab:orange', alpha=0.8)

        axes[i][j].set_title(('$\\beta$-spectrum of $^{14}$C isotope\n' +
                              'One of the 2 minute measurements (index : {0})'.format(random_indeces[i*ncols + j])),
                             fontsize=titlesize, fontweight='bold')
        axes[i][j].set_xlabel('Light output [keVee]', fontsize=axislabelsize)
        axes[i][j].set_ylabel('Counts [N]', fontsize=axislabelsize)
        axes[i][j].tick_params(axis='both', which='major', labelsize=axisticksize)

plt.savefig(figsave_dir + 'random_spectrums.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))

titlesize = 25
axislabelsize = 25
axisticksize = 20

axes.plot(data.index, data['mean']/np.sum(data['mean']),
          lw=4, color='tab:orange', alpha=0.8)

axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nMean values of all 2 minute measurements',
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.savefig(figsave_dir + 'mean_spectrum.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*20,nrows*14))

titlesize = 25
axislabelsize = 25
axisticksize = 20

x_lim = (-12.5, 262.5)
x_ticks_loc = axes.get_xticks()
x_ticks_val = [int(x*(np.max(data.index.values)+0.25)) for x in x_ticks_loc]

sns.barplot(data.index, y='mean', data=data,
            palette='autumn', ci=None,
            ax=axes)

#axes.bar(data.index, data['mean']/np.sum(data['mean']),
#         width=0.5, align='center', color='tab:orange', alpha=1.0)

axes.set_xlim(x_lim)
axes.set_xticks(list(np.array(x_ticks_val)*2))
axes.set_xticklabels(x_ticks_val)
axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nMean values of all 2 minute measurements',
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.savefig(figsave_dir + 'mean_spectrum_binned.' + 'png',
            format='png',
            dpi=300,
            bbox_inches='tight')
plt.show()

### Calculate CPM

In [ ]:
CPM = (0.5 * data['mean'].values).sum() / 2
eta = CPM / DPM
print(b1 + 'Measured CPM : ' + b0 + '{0:.0f}'.format(CPM))
print(b1 + 'Measured eta : ' + b0 + '{0:.3f}%'.format(eta*100))

### Calculate wheter if $\left< E \right> = \frac{Q}{3}$

In [ ]:
E_pred = (data.index.values * data['mean'].values).sum()/data['mean'].sum()
E_grtr = Q/3
diff = np.abs(E_pred - E_grtr)

print(b1 + 'Difference from groundtruth value is ' + b0 + '{0:.2f} %'.format(diff/E_grtr * 100))

In [ ]:
E_grtr

In [ ]:
E_pred

### Test wheter if $\sqrt{N} \approx \sigma$

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*12,nrows*12))

titlesize = 25
axislabelsize = 24
axisticksize = 20
labelpad = 15

std_1 = np.sqrt(data['mean'].values)
std_2 = data['std'].values

norm = np.max([std_1, std_2])

axes.scatter(std_1/norm, std_2/norm,
             color='tab:blue', s=60, alpha=0.75)
axes.plot([0,1],[0,1], color='tab:red', lw=6, alpha=0.7)

rmse = np.sqrt(mean_squared_error(std_1/norm, std_2/norm))

axes.set_title('Comparsion of $\sqrt{N}$ and $\sigma$\n' + 'RMSE = {0:.3f}'.format(rmse),
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Square root of the mean measured\nvalues $\\left( \\sqrt{N} \\right)$ for a specific bin',
                fontsize=axislabelsize, labelpad=labelpad)
axes.set_ylabel('Standard deviation $\\left( \\sigma \\right)$ of\nmeasurements for a specific bin',
                fontsize=axislabelsize, labelpad=labelpad)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.savefig(figsave_dir + 'error_compare.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

## Fit theoretical curves on spectrums
### Non-relat without Fermi function

In [ ]:
def fit_non_relat(E, A, B, C, D):
    
    return A + B * E**(2/3) - C * E**(5/2) + D * E**(7/2)

In [ ]:
energies = data.index
popt_nrel, pcov_nrel = curve_fit(fit_non_relat, energies, data['mean'])
perr_nrel = np.sqrt(np.diag(pcov_nrel))
print(perr_nrel / popt_nrel * 100)

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))

titlesize = 25
axislabelsize = 25
axisticksize = 20
legendsize = 20

axes.plot(data.index, data['mean'], label='Measured spectrum',
          lw=4, color='tab:orange', alpha=1.0)
axes.plot(energies, fit_non_relat(energies, *popt_nrel), label='Theoretical fitted',
          lw=3, color='tab:blue', ls='--', alpha=0.8)


axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nNon-relativistic model', fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.legend(fontsize=legendsize)
plt.savefig(figsave_dir + 'non_relat_fit.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

### Non-relat with Fermi function

In [ ]:
def fit_non_relat_fermi(E, p1, p2, p3, p4, p5):
    """
    Non-relativistic case. The Fermi function is the following:
    ``F = 2*pi*eta / (1 - e^(-2*pi*eta))``
    
    Where eta is p-dependent variable:
    ``eta = Z*e^2*c / (h/2pi * p)``
    Where `Z` is the beta-decayed nuclei's proton number, `e` is the
    elemental charge, `c` is the speed of light, `h` is the Plack
    constant, `p` is the momentum of the beta-particle.
    
    The momentum `p` could be evaluated by the known formula:
    ``p = sqrt(2*m*E)``
    Where `m` is the electron mass
    """
    
    # mass of electron [kg]
    m_e = 9.10938356e-31
    # light speed [m/s]
    c_light = 299792458
    
    # Something:
    #p = np.sqrt((E/c_light)**2 - (m_e * c_light)**2)
    # Non-realtivistic:
    p = np.sqrt(2*m_e*E)
    
    # Calculating Fermi function
    # --------------------------
    # eta = Z*e**2 * c / h_bar * p
    # e**2 * c / h_bar = 72974 J/(T^2 * m^3) (joules per tesla squared cubic meter)
    # eta = Z * 72974 / p
    # [eta] = (J/(T^2 * m^3)) / (kg * m/s)
    # [eta] = 1 / (s*(T^2 * m^2))
    # Since we measured 14C : Z = 6
    eta = Z * 72974 / p
    F = 2*np.pi*eta/(1 - np.e**(-2*np.pi*eta))

    return p1 + (p2 + p3 * E**(2/3) - p4 * E**(5/2) + p5 * E**(7/2)) * F

In [ ]:
energies = data.index
popt_nrel, pcov_nrel = curve_fit(fit_non_relat_fermi, energies, data['mean'])
perr_nrel = np.sqrt(np.diag(pcov_nrel))
print(perr_nrel / popt_nrel * 100)

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))

titlesize = 25
axislabelsize = 25
axisticksize = 20
legendsize = 20

axes.plot(data.index, data['mean'], label='Measured spectrum',
          lw=4, color='tab:orange', alpha=1.0)
axes.plot(energies, fit_non_relat_fermi(energies, *popt_nrel), label='Theoretical fitted',
          lw=3, color='tab:blue', ls='--', alpha=0.7)

axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nNon-relativistic model with Fermi function',
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.legend(fontsize=legendsize)
plt.savefig(figsave_dir + 'non_relat_fermi_fit.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

In [ ]:
def fit_relat_fermi(E, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10,
                    p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, Z, omega):
    """
    Relativistic case. The Fermi function is the following:
    ``F = 2*pi*eta / (1 - e^(-2*pi*eta))``
    
    Where eta is p-dependent variable:
    ``eta = Z*e^2*c / (h/2pi * p)``
    Where `Z` is the beta-decayed nuclei's proton number, `e` is the
    elemental charge, `c` is the speed of light, `h` is the Plack
    constant, `p` is the momentum of the beta-particle.
    
    The momentum `p` could be evaluated by the known formula:
    ``p = sqrt((E + m*c^2)^2 - (m*c^2)^2)``
    Where `m` is the electron mass
    """

    # Relativistic E_tot(E) and p(E)
    E_tot = m_e*c_light**2 + E
    p = np.sqrt((E_tot)**2 - m_e**2 * c_light**4)

    # Calculating Fermi function
    # --------------------------
    # eta = Z*e**2 * c / h_bar * p
    # e**2 * c / h_bar = 72974 J/(T^2 * m^3) (joules per tesla squared cubic meter)
    # eta = Z * 72974 / p
    # [eta] = (J/(T^2 * m^3)) / (kg * m/s)
    # [eta] = 1 / (s*(T^2 * m^2))
    # Since we measured 14C : Z = 6
    eta = Z * 72974 / p
    F = 2*np.pi*eta/(1 - np.e**(-2*np.pi*eta))
    
    return p1 + p2 * F * \
           (1/60 * c_light * np.sqrt(E * (2 * c_light**2 * m_e + p3 + p4 * E)) * \
            (-15 * c_light**8 * m_e**4 + 5 * c_light**6 * m_e**3 * (p5 + p6 * E - 3 * Q) + \
             c_light**4 * m_e**2 * E * (5 * Q - 2 * E * p7 + p8) * p8 + \
             2 * c_light**2 * m_e * E * (20 * Q**2 - p10 * 25 * Q * E + p11 + 9 * p12 * E**2) + \
             (30 * c_light**8 * m_e**4 * (c_light**2 * m_e + Q) * \
              np.log(np.sqrt(2 * c_light**2 * m_e + E * p13 + p14) + np.sqrt(E*p15 + p16)))/ \
             (np.sqrt(E*p17 + p18) * np.sqrt(2 * c_light**2 * m_e + E*p19 + p20)) + \
             p21 * 2 * E**2 * (10 * Q**2 - 15 * Q * E * p22 + Z * 6 * E**2 + omega)
            )
           ) + p23*F

In [ ]:
energies = data.index
popt_rel, pcov_rel = curve_fit(fit_relat_fermi, energies, data['mean'])
perr_rel = np.sqrt(np.diag(pcov_rel))
print(perr_rel / popt_rel * 100)

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))

titlesize = 25
axislabelsize = 25
axisticksize = 20
legendsize = 20

axes.plot(data.index, data['mean'], label='Measured spectrum',
          lw=4, color='tab:orange', alpha=1.0)
axes.plot(energies, fit_relat_fermi(energies, *popt_rel), label='Theoretical fitted',
          lw=3, color='tab:blue', ls='--', alpha=0.8)

axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nRelativistic model with Fermi function',
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.legend(fontsize=legendsize)
plt.savefig(figsave_dir + 'relat_fermi_fit.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()

In [ ]:
def relat_fermi_fit2(E, p1, p2, p3, p4, p5, p6,
                     p7, p8, p9, p10, p11, p12, p13):
    
    # Relativistic E_tot(E) and p(E)
    E_tot = m_e*c_light**2 + E
    p = np.sqrt((E_tot)**2 - m_e**2 * c_light**4)

    # Calculating Fermi function
    # --------------------------
    # eta = Z*e**2 * c / h_bar * p
    # e**2 * c / h_bar = 72974 J/(T^2 * m^3) (joules per tesla squared cubic meter)
    # eta = Z * 72974 / p
    # [eta] = (J/(T^2 * m^3)) / (kg * m/s)
    # [eta] = 1 / (s*(T^2 * m^2))
    # Since we measured 14C : Z = 6
    eta = Z * 72974 / p
    F = 2*np.pi*eta/(1 - np.e**(-2*np.pi*eta))
    
    return (np.sqrt(p1 * E**2 + p2 * E + p3) *
           (p4 * E**3 + p5 * E**2 + p6 * E) -
           (p7 * np.arcsinh(p8 * np.sqrt(E) + p9) + p10)) * F * p11 + p12 * F + p13

In [ ]:
energies = data.index
popt_rel, pcov_rel = curve_fit(relat_fermi_fit2, energies, data['mean'])
perr_rel = np.sqrt(np.diag(pcov_rel))
print(perr_rel / popt_rel * 100)

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*14,nrows*8))

titlesize = 25
axislabelsize = 25
axisticksize = 20
legendsize = 20

axes.plot(data.index, data['mean'], label='Measured spectrum',
          lw=4, color='tab:orange', alpha=1.0)
axes.plot(energies, relat_fermi_fit2(energies, *popt_rel), label='Theoretical fitted',
          lw=3, color='tab:blue', ls='--', alpha=0.8)

axes.set_title('$\\beta$-spectrum of $^{14}$C isotope\nRelativistic model with Fermi function',
               fontsize=titlesize, fontweight='bold')
axes.set_xlabel('Light output [keVee]', fontsize=axislabelsize)
axes.set_ylabel('Probability', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.legend(fontsize=legendsize)
plt.savefig(figsave_dir + 'relat_fermi_fit2.' + figsave_format,
            format=figsave_format,
            dpi=figsave_dpi,
            bbox_inches='tight')
plt.show()